# FFCSA demo examples

## Loading packages

In [1]:
LoadPackage("fsr");

misc.gd OK,	outputs.gd OK,	fsr.gd OK,	lfsr.gd OK,	nlfsr.gd OK,	filfun.gd OK,	o\
utfsr.gd OK,	drawlfsr.gd OK,	drawnlfsr.gd OK,	init.g done!!!
misc.gi OK,	outputs.gi OK,	fsr.gi OK,	lfsr.gi OK,	nlfsr.gi OK,	fsrfil.gi OK,	o\
utfsr.gi OK,	drawlfsr.gi OK,	drawnlfsr.gi OK,	read.g done!!!

You can now use 200 variables x_0 ...x_199


true

In [2]:
LoadPackage("ffcsa");

misc.gd OK,	weight.gd OK,	matrix.gd OK,	nb.gd OK,	findElm.gd OK,	findPoly.gd O\
K,	ffbases.gd  OK,	findElm.gd OK,	findPolySpecial.gd OK,	algs.gd OK,	algsmatri\
x.gd OK,	init.g done!!!
misc.gi OK,	weight.gi OK,	matrix.gi OK,	nb.gi OK,	findElm.gi OK,	findPoly.gi O\
K,	ffbases.gi  OK,	findElmSpecial.gi OK,	findPolySpecial.gi OK,	
variables
[ "a_0", "a_1", "a_2", "a_3", "a_4", "a_5", "a_6", "a_7", "a_8", "a_9" ]
[ "b_0", "b_1", "b_2", "b_3", "b_4", "b_5", "b_6", "b_7", "b_8", "b_9" ]
[ "d_0", "d_1", "d_2", "d_3", "d_4", "d_5", "d_6", "d_7", "d_8", "d_9", 
  "d_10", "d_11", "d_12", "d_13", "d_14", "d_15", "d_16", "d_17", "d_18" ]
algs.gi OK,	algsmatrix.gi OK,	read.g done!!!


true

## Generating Multiplication Expressions for ${\mathbb F}_{2^4}$
 

We use irreducible polynomial $f(x)=x^4+x+1$ with root $\rho\in{\mathbb F}_{2^4}$ and the polynomial basis 
${\rm PB}=\{1, \rho, \rho^2, \rho^3\}$. 
The GAP code below shows the setup


In [7]:
K := GF(2);; x := X(K, "x");;
f := x^4+x+1;; F := FieldExtension(K,  f);;
PB := GeneratePB(F, RootOfDefiningPolynomial(F));


Basis( GF(2^4), [ Z(2)^0, Z(2^4), Z(2^4)^2, Z(2^4)^3 ] )

The `ChooseFieldElms` method prepares vectors $avec=\left[a_0, \dots , a_{m-1}\right]$ and 
$bvec=\left[b_0,\dots,b_{m-1}\right]$ with default direction `To`, where $m=[{\mathbb F}_{2^4}:{\mathbb F}_{2}]$ is the degree of extension.
Note that $a_i, b_j$ are not coefficients, but 
[GAP variables](https://docs.gap-system.org/doc/ref/chap4_mj.html#X7A4C2D0E7E286B4F) 
to allow symbolic computation. 
 

In [8]:
ChooseFieldElms(F);


variables
[ "a_0", "a_1", "a_2", "a_3" ]
[ "b_0", "b_1", "b_2", "b_3" ]
[ "d_0", "d_1", "d_2", "d_3", "d_4", "d_5", "d_6" ]


Next we generate the multiplication matrix ${\rm U}$ from basis ${\rm PB}$ and variables in $avec$

In [10]:
U := MatrixUExpression(PB, avec);
for i in U do Display(i); od;

[ a_0, a_3, a_2, a_1 ]
[ a_1, a_0+a_3, a_2+a_3, a_1+a_2 ]
[ a_2, a_1, a_0+a_3, a_2+a_3 ]
[ a_3, a_2, a_1, a_0+a_3 ]


[ [ a_0, a_3, a_2, a_1 ], [ a_1, a_0+a_3, a_2+a_3, a_1+a_2 ],   [ a_2, a_1, a_0+a_3, a_2+a_3 ], [ a_3, a_2, a_1, a_0+a_3 ] ]

The `FFA_mult_matrixU` method produces the expressions used for the hardware implementation
by multiplying ${\rm U}$ and $bvec$
 

In [12]:
mult := FFA_mult_matrixU(PB, avec, bvec);; 
for i in mult do Display(i); od; 

a_0*b_0+a_1*b_3+a_2*b_2+a_3*b_1
a_0*b_1+a_1*b_0+a_1*b_3+a_2*b_2+a_2*b_3+a_3*b_1+a_3*b_2
a_0*b_2+a_1*b_1+a_2*b_0+a_2*b_3+a_3*b_2+a_3*b_3
a_0*b_3+a_1*b_2+a_2*b_1+a_3*b_0+a_3*b_3


For example, to drive the multiplier output $c_0$, 
the expression $a_0b_0+a_1b_3+a_2b_2+a_3b_1$ 
must be implemented in hardware. 

## Generating Multiplication Expressions for ${\mathbb F}_{((2^2)^2)^2}/{\mathbb F}_{(2^2)^2}$


In this example we use the tower field ${\mathbb F}_2 \xrightarrow{f_1(x)} {\mathbb F}_{2^2} \xrightarrow{f_2(x)} {\mathbb F}_{(2^2)^2} \xrightarrow{f_3(x)} {\mathbb F}_{((2^2)^2)^2} $ with defining polynomials 
- $f_1(x)=x^2 + x + 1$ with root $\lambda$, i.e., $f_1(\lambda)=0$ 
- $f_2(x)=x^2 + \lambda x + 1$ with root $\mu$, i.e., $f_2(\mu)=0$ 
- $f_3(x)=x^2 + \lambda x + \lambda^2\mu$ with root $\nu$, i.e. $f_3(\nu)=0$ 	

and polynomial bases on each extension. 
	

In [18]:
f1 := x^2+x+Z(2)^0;;         F1 := FieldExtension(K,  f1);;
f2 := x^2+Z(2^2)*x+Z(2)^0;;  F2 := FieldExtension(F1, f2);; 
f3 := x^2+Z(2^2)*x+Z(2^4);;  F3 := FieldExtension(F2, f3);; 

First we need to prepare the variables. Note that `ChooseFieldElms(F3)` returns  vectors of length 2, not 8.

In [19]:
ChooseFieldElms(F3); 


variables
[ "a_0", "a_1" ]
[ "b_0", "b_1" ]
[ "d_0", "d_1", "d_2" ]


The input to method `FFA_mult_matrixU` is the *per-level* polynomial basis `B3`$=\{1,\nu\}$, obtained for
${{\mathbb F}_{((2^2)^2)^2}/{\mathbb F}_{(2^2)^2}}$. 

In [21]:
nu := RootOfDefiningPolynomial(F3);;
B3 := GeneratePB(F3, nu); 

Basis( AsField( AsField( GF(2^2), GF(2^4) ), GF(2^8) ), [ Z(2)^0, Z(2^8)^76  ] )

In [23]:
multB3 := FFA_mult_matrixU(B3, avec, bvec);; 
for i in multB3 do 
    Display(i); 
od; 

a_0*b_0+Z(2^4)*a_1*b_1
a_0*b_1+a_1*b_0+Z(2^2)*a_1*b_1


Method `FFA_mult_matrixU` produces the expressions for two outputs of the multiplier. 
This multiplication is on the top level of the tower field ${{\mathbb F}_{((2^2)^2)^2}/{\mathbb F}_{(2^2)^2}}$.
It contains submodules: multipliers on the lower level ${{\mathbb F}_{(2^2)^2}}/{\mathbb F}_{2^2}$.

We also need two subfield constant multipliers for $\gamma_1=\lambda^2\mu\in{\mathbb F}_{(2^2)^2}$
and $\gamma_2=\lambda\in{\mathbb F}_{(2^2)^2}$. Although $\lambda\in{\mathbb F}_{2^2}$, the product
$a_1b_1\in{\mathbb F}_{(2^2)^2}$, therefore we must perform $\times\gamma_2$
in ${\mathbb F}_{(2^2)^2}/{\mathbb F}_{2^2}$. 


In [27]:
lambda := RootOfDefiningPolynomial(F1);;
mu := RootOfDefiningPolynomial(F2);; 
lambda^2*mu;
lambda; 

Z(2^4)

Z(2^2)

## Extracting submodules

We parse the top-level expressions to extract submodules. 
First we ensure all exponents are reduced, then we split the expression into two vectors:
- monomials $\rightarrow $ finite field multiplications and exponentiations
- coefficients $\rightarrow $ multiplicative and additive constants 

Note that the *new* top-level is  ${\mathbb F}_{(2^2)^2}$: we already generated the expressions for 
arithmetic in ${\mathbb F}_{((2^2)^2)^2}$ and moved a level down to ${\mathbb F}_{(2^2)^2}$


In [29]:
expr := multB3[1];; 
SplitCoeffsAndMonomials(F2,expr);

[ [ Z(2)^0, Z(2^4) ], [ a_0*b_0, a_1*b_1 ] ]

## Test vectors for tower field constructions

FFCSA can generate different tower-field bases (\eg\ NB on first and PB on second extension). To continue the example in this demo, we use a basis of ${{\mathbb F}_{2^8}/{\mathbb F}_2}$, obtained as products of *per-level* basis elements
 ${\rm B}_{{\mathbb F}_{2^8}/{\mathbb F}_2}=\{t_0, t_1, \dots , t_7\}
=\{1, \lambda, \mu, \mu\lambda,\nu, \nu\lambda, \nu\mu, \nu\mu\lambda\}.$


In [32]:
Mlist := [["PB", "to"], ["PB", "to"], ["PB", "to"]];
TFB2 := GenerateTFBfromEDPLwithMB([f1, f2, f3 ] , Mlist);
nu*mu*lambda;

[ [ "PB", "to" ], [ "PB", "to" ], [ "PB", "to" ] ]

Basis( GF(2^8), [ Z(2)^0, Z(2^2), Z(2^4)^6, Z(2^4)^11, Z(2^8)^76, Z(2^8)^161,   Z(2^8)^178, Z(2^8)^8 ] )

Z(2^8)^8

We can now generate test-vectors for verification of the multiplier:

In [38]:
a := Z(2^8)^15;;
b := Z(2^8)^5;;
c := a*b; 
VecToString(Coefficients(TFB2, a));
VecToString(Coefficients(TFB2, b));
VecToString(Coefficients(TFB2, c)); 

Z(2^8)^20

"11101010"

"00101110"

"10001010"